In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from os import listdir

from torchvision import transforms, utils
from skimage import io, transform

import pandas as pd 
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import glob

In [3]:
df=pd.read_csv('data/artists.csv')

In [5]:
df.head()

,id,name,years,genre,nationality,bio,wikipedia,paintings
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194


In [6]:
len(df.name)

50

In [10]:
mypath = 'data/resized'

In [11]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [12]:
onlyfiles

['Albrecht_DuÔòá├¬rer_1.jpg',
 'Albrecht_DuÔòá├¬rer_10.jpg',
 'Albrecht_DuÔòá├¬rer_100.jpg',
 'Albrecht_DuÔòá├¬rer_101.jpg',
 'Albrecht_DuÔòá├¬rer_102.jpg',
 'Albrecht_DuÔòá├¬rer_103.jpg',
 'Albrecht_DuÔòá├¬rer_104.jpg',
 'Albrecht_DuÔòá├¬rer_105.jpg',
 'Albrecht_DuÔòá├¬rer_106.jpg',
 'Albrecht_DuÔòá├¬rer_107.jpg',
 'Albrecht_DuÔòá├¬rer_108.jpg',
 'Albrecht_DuÔòá├¬rer_109.jpg',
 'Albrecht_DuÔòá├¬rer_11.jpg',
 'Albrecht_DuÔòá├¬rer_110.jpg',
 'Albrecht_DuÔòá├¬rer_111.jpg',
 'Albrecht_DuÔòá├¬rer_112.jpg',
 'Albrecht_DuÔòá├¬rer_113.jpg',
 'Albrecht_DuÔòá├¬rer_114.jpg',
 'Albrecht_DuÔòá├¬rer_115.jpg',
 'Albrecht_DuÔòá├¬rer_116.jpg',
 'Albrecht_DuÔòá├¬rer_117.jpg',
 'Albrecht_DuÔòá├¬rer_118.jpg',
 'Albrecht_DuÔòá├¬rer_119.jpg',
 'Albrecht_DuÔòá├¬rer_12.jpg',
 'Albrecht_DuÔòá├¬rer_120.jpg',
 'Albrecht_DuÔòá├¬rer_121.jpg',
 'Albrecht_DuÔòá├¬rer_122.jpg',
 'Albrecht_DuÔòá├¬rer_123.jpg',
 'Albrecht_DuÔòá├¬rer_124.jpg',
 'Albrecht_DuÔòá├¬rer_125.jpg',
 'Albrecht_DuÔòá├¬rer_126.jpg',
 'Albrecht_Du

In [13]:
import re
def get_numbers_from_filename(filename):
    return re.search(r'\d+', filename).group(0)

In [14]:
num_list=[]
for filename in onlyfiles:
    num_list.append(get_numbers_from_filename(filename))

In [15]:
num_list

['1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '2',
 '20',
 '200',
 '201',
 '202',
 '203',
 '204',
 '205',
 '206',
 '207',
 '208',
 '209',
 '21',
 '210',
 '211',
 '212',


In [16]:
data_list=[]
for name in tqdm(df.name.tolist()):
    for file in onlyfiles:
        if all(x in file for x in name.split(' ')):
            data_list.append([name,get_numbers_from_filename(file),f'{mypath}/{file}'])
  

100%|██████████| 50/50 [00:00<00:00, 273.75it/s]


In [17]:
df_bilder=pd.DataFrame(data=data_list,columns=['name','num','path'])

In [18]:
df=df_bilder.merge(df,on='name')

In [39]:
image_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=64),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop(size=64),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
valid_transforms = transforms.Compose([
    transforms.Resize(size=64),
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop(size=64),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [69]:
train_pics_list = []
for path in tqdm(df.path):
    img = Image.open(path)
    img_tensor = image_transforms(img)
    #img_tensor = img_tensor.unsqueeze(0)
    train_pics_list.append(img_tensor)
df["tensor"] = train_pics_list
df.drop(["years", "bio", "nationality", "genre", "wikipedia", "num", "name", "paintings"], axis=1, inplace=True)

100%|██████████| 8027/8027 [00:32<00:00, 243.45it/s]


KeyError: "['years', 'bio', 'nationality', 'genre', 'wikipedia', 'num', 'name', 'paintings'] not found in axis"

In [70]:
df.head(3)

,path,id,tensor
0,data/resized/Amedeo_Modigliani_1.jpg,0,"[[[tensor([-2.1179, -2.1179, 0.1939, -0.2342,..."
1,data/resized/Amedeo_Modigliani_10.jpg,0,"[[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,..."
2,data/resized/Amedeo_Modigliani_100.jpg,0,"[[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,..."


In [71]:
y=df.id.tolist()
X=df.tensor.tolist()

In [72]:
random_seed=42
test_size=0.3

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, random_state=random_seed, test_size=test_size)

In [73]:
X_train_all

[tensor([[[[ 0.6049,  0.5878,  0.5536,  ...,  0.6049,  0.6221,  0.6563],
           [ 0.5536,  0.5536,  0.5364,  ...,  0.6392,  0.6392,  0.6563],
           [-0.1657,  0.5193,  0.6049,  ...,  0.6906,  0.6734,  0.6906],
           ...,
           [-1.0219, -1.0562, -1.0904,  ...,  1.1700,  1.9749,  1.9749],
           [-1.1418, -1.1247, -1.0562,  ...,  1.1358,  1.9578,  2.0092],
           [-1.2445, -1.1760, -0.6109,  ...,  1.0673,  1.9235,  2.0263]],
 
          [[ 0.7479,  0.7304,  0.6954,  ...,  0.7479,  0.7654,  0.8004],
           [ 0.6954,  0.6954,  0.6779,  ...,  0.7829,  0.7829,  0.8004],
           [-0.0399,  0.6604,  0.7479,  ...,  0.8354,  0.8179,  0.8354],
           ...,
           [-0.9153, -0.9503, -0.9853,  ...,  1.3256,  2.1485,  2.1485],
           [-1.0378, -1.0203, -0.9503,  ...,  1.2906,  2.1310,  2.1835],
           [-1.1429, -1.0728, -0.4951,  ...,  1.2206,  2.0959,  2.2010]],
 
          [[ 0.9668,  0.9494,  0.9145,  ...,  0.9668,  0.9842,  1.0191],
           [ 

In [47]:
from torchvision import models

In [74]:
df.id.unique().tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [94]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = torch.tensor(y).float()

    def __len__(self):
        return len(self.x)

    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return {'x': x, 'y': y}
    
BATCH_SIZE = 64    
train_dataset = MyDataset(X_train_all, y_train_all)
dataLoader_train = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True)

test_dataset = MyDataset(X_test_all, y_test_all)
dataLoader_test = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=BATCH_SIZE,
                                              shuffle=True)

In [95]:
dataloaders = {
    'train': torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=True),
    'valid': torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
}

In [98]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(df.id.unique().tolist()))

c:\Users\nikla\anaconda3\envs\NLP\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nikla\anaconda3\envs\NLP\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [76]:
from torchsummary import summary
from torch.optim import lr_scheduler

In [77]:
print('Model Summary:-\n')
for num, (name, param) in enumerate(model.named_parameters()):
    print(num, name, param.requires_grad)
summary(model, input_size=(3, 64, 64))
print(model)

Model Summary:-

0 conv1.weight True
1 bn1.weight True
2 bn1.bias True
3 layer1.0.conv1.weight True
4 layer1.0.bn1.weight True
5 layer1.0.bn1.bias True
6 layer1.0.conv2.weight True
7 layer1.0.bn2.weight True
8 layer1.0.bn2.bias True
9 layer1.1.conv1.weight True
10 layer1.1.bn1.weight True
11 layer1.1.bn1.bias True
12 layer1.1.conv2.weight True
13 layer1.1.bn2.weight True
14 layer1.1.bn2.bias True
15 layer2.0.conv1.weight True
16 layer2.0.bn1.weight True
17 layer2.0.bn1.bias True
18 layer2.0.conv2.weight True
19 layer2.0.bn2.weight True
20 layer2.0.bn2.bias True
21 layer2.0.downsample.0.weight True
22 layer2.0.downsample.1.weight True
23 layer2.0.downsample.1.bias True
24 layer2.1.conv1.weight True
25 layer2.1.bn1.weight True
26 layer2.1.bn1.bias True
27 layer2.1.conv2.weight True
28 layer2.1.bn2.weight True
29 layer2.1.bn2.bias True
30 layer3.0.conv1.weight True
31 layer3.0.bn1.weight True
32 layer3.0.bn1.bias True
33 layer3.0.conv2.weight True
34 layer3.0.bn2.weight True
35 layer3.0.b

In [78]:
criterion = nn.CrossEntropyLoss()
# Optimizer 
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Learning rate decay
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [99]:
import copy
from torch.utils.tensorboard import SummaryWriter

print("\nTraining:-\n")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    #since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
# Tensorboard summary
    writer = SummaryWriter()
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
# Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                running_loss = 0.0
                running_corrects = 0
# Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs
                labels = labels
                optimizer.zero_grad()
# forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
# backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
# statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / 1
            epoch_acc = running_corrects.double() / 1

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            # Record training loss and accuracy for each phase
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            print('Best val Acc: {:4f}'.format(best_acc))
            # load best model weights
            model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=10)
# Save the model
PATH="model_1.pth" 
print("\nSaving the model...")
torch.save(model_ft, PATH)


Training:-

Epoch 0/9
----------


TypeError: conv2d() received an invalid combination of arguments - got (str, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)


In [68]:
X_train_all[1]

tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1